## **Import Relevant Packages**

In [ ]:
# Install Package

pip install feature_engine

In [ ]:
# Import Packages

import os
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
from IPython.display import Image
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from feature_engine.encoding import OrdinalEncoder

# 1. Pre-Processing Data

### 1. Import Dataset

In [ ]:
# Import data training (id, label)
data_train = pd.read_csv("../input/currency/labels_train_hp.csv")
data_train.head()

# Import data testing (id, label)
data_test = pd.read_csv("../input/currency/labels_test_hp.csv")
data_test.head()

### 2. Change ID to File Path

In [ ]:
# Change id to file path (TRAIN)

file_path = []
for x in data_train["path"]:
    file_path.append(f"../input/currency/train/train/{x}")

data_train["path"] = file_path
data_train = data_train[["path", "label"]]
data_train.head()

In [ ]:
# Change id to file path (TEST)

file_path = []
for x in data_test["path"]:
    file_path.append(f"../input/currency/test/test/{x}")

data_test["path"] = file_path
data_test = data_test[["path", "label"]]
data_test.head()

### 3. Turn Labels (y) into Number (Tensors)

In [ ]:
# Prepare Labels
labels_train = np.array(data_train['label'])
labels_test = np.array(data_test['label'])

# Unique Labels
unique_labels = np.unique(labels_train)
unique_labels

In [ ]:
# Turn each label into an array of booleans (TRAIN)
boolean_labels_train = [label == unique_labels for label in labels_train]

# Turn each label into an array of booleans (TEST)
boolean_labels_test = [label == unique_labels for label in labels_test]

In [ ]:
# Total label
len(boolean_labels_train), len(boolean_labels_test)

### 4. Define Train & Validation Set

In [ ]:
# TRAIN
X_train = data_train["path"]
y_train = boolean_labels_train

# VALIDATION
X_val = data_test['path']
y_val = boolean_labels_test

In [ ]:
# Total data of training and validation set

print("Train Set:", len(X_train))
print("Validation Set:", len(X_val))

### 5. Turn Images (X) into Number (Tensors)

In [ ]:
# Train set
train_set = tf.data.Dataset.from_tensor_slices((tf.constant(X_train),
                                                tf.constant(y_train)))
# Validation Set
val_set = tf.data.Dataset.from_tensor_slices((tf.constant(X_val),
                                              tf.constant(y_val)))

#### 6. Shuffling Train Set


In [ ]:
# Shuffle train set

BUFFER_SIZE = len(train_set)
train_set = train_set.shuffle(buffer_size=BUFFER_SIZE)

#### 7. Function for Preprocessing Data

In [ ]:
# Function for process image

def process_image(image_path):
    # Read an image
    image = tf.io.read_file(image_path)
    
    # Turn jpeg to numbers (isinya probability color 0-255)
    image = tf.image.decode_jpeg(image, channels=3)

    # Convert the color channel values from 0-255 to 0-1
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    image /= 255
    
    # Resize image to our desired shape (height,width)
    image = tf.image.resize(image, size=[480,640])

    return image

In [ ]:
# Function to get tuple of the dataset

def get_tuple(image_path, label):
    # Pre-process dulu sama function yg tadi
    image = process_image(image_path)
    
    # Return tuple
    return image, label

In [ ]:
# Apply function to train set
train_set = train_set.map(get_tuple)

# Apply function to validation set
val_set = val_set.map(get_tuple)

### 8. Turn Data into Batches

In [ ]:
BATCH_SIZE = 1

# train set
train_set = train_set.batch(BATCH_SIZE)

# validation set
val_set = val_set.batch(BATCH_SIZE)

In [ ]:
# Check element spec
train_set.element_spec, val_set.element_spec

## **2. Building The ANN Model**


### 1. Create Model

In [ ]:
# Setup the input shape
INPUT_SHAPE = [None, 390, 828, 3] #batch, height, width, colour channels

# Setup the output shape
OUTPUT_SHAPE = len(unique_labels)

In [ ]:
# Initialize model

tf.keras.applications.mobilenet_v2.trainable = False
model = tf.keras.Sequential([
        tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape = (390, 828, 3), weights = None, include_top = False),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(60),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10),
        tf.keras.layers.Dense(units=OUTPUT_SHAPE, activation = "softmax")],
        name = "MobileNetV2_Categorical_Classification")

model.compile(optimizer = 'adam',
                  loss = "categorical_crossentropy",
                  metrics = ["accuracy"])

In [ ]:
# View the structure of the model
model.summary()

### 2. Train Model

In [ ]:
history = model.fit(train_set,
                    epochs = 3,
                    validation_data = val_set,
                    verbose = 2)

## **3. Making Predictions**

In [ ]:
# Make predictions on validation set
predictions = model.predict(val_set)

In [ ]:
# Turn prediction probabilities into prediction labels
def get_pred_label(prediction_probabilities):
    return unique_labels[np.argmax(prediction_probabilities)]

# Example
pred_label = get_pred_label(predictions[2])
pred_label

## **4. Saving & Reloading Trained Model**

In [ ]:
# Create a function that save the model
def save_model(model, suffix=None):
    # Read model directory
    modeldir = "./"
    
    # Format path model
    model_path = f"./{suffix}.h5"

    print(f"Saving model to: {model_path}")
    model.save(model_path)
    return print("Model saved!")

In [ ]:
# Save our model trained on 1,000 images
save_model(model, suffix="mobilenet_crop_hp")